In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    # --- 【请修改】输入和输出路径 ---
    INPUT_FILE = Path(r'E:\工作簿3.xlsx')
    OUTPUT_FILE = Path(r'E:\工作簿3_result.xlsx')

    # --- Sheet名称 ---
    SHEET_2025 = '2025'
    SHEET_2024 = '2024'

    # --- 关键列名 ---
    DATE_COL = 'ds'
    INDUSTRY_NAME_COL = 'code_cls_val_name'

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def get_load_columns() -> tuple[list[str], list[str]]:
    """程序化地生成所有负荷列名和峰时段负荷列名"""
    all_load_cols = []
    peak_load_cols = []
    for hour in range(24):
        for minute in range(0, 60, 15):
            time_str = f"{hour:02d}{minute:02d}"
            col_name = f"load_{time_str}"
            all_load_cols.append(col_name)
            # 判断是否在9:00到15:00的峰时段
            if 9 <= hour <= 15:
                peak_load_cols.append(col_name)
    return all_load_cols, peak_load_cols

def calculate_peak_ratio(df: pd.DataFrame, all_load_cols: list[str], peak_load_cols: list[str]) -> pd.DataFrame:
    """
    计算给定DataFrame中每一行的峰时段负荷占比。
    """
    print(f"  -> 正在计算峰时段负荷占比...")
    
    # 1. 确保所有负荷列都是数值类型，非数值转为NaN，然后用0填充
    df[all_load_cols] = df[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # 2. 计算总负荷和峰时段负荷
    total_load = df[all_load_cols].sum(axis=1)
    peak_load = df[peak_load_cols].sum(axis=1)
    
    # 3. 计算占比，并处理总负荷为0的情况（防止除零错误）
    #    使用 np.where: 如果 total_load > 0，则正常计算；否则，占比为0。
    df['peak_ratio'] = np.where(total_load > 0, peak_load / total_load, 0)
    
    print("  -> 计算完成。")
    return df

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"--- 开始处理文件: {Config.INPUT_FILE} ---")

    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    try:
        # 1. 加载数据
        print(f"正在加载Sheet '{Config.SHEET_2025}'...")
        df_2025 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2025)
        
        print(f"正在加载Sheet '{Config.SHEET_2024}'...")
        df_2024 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2024)

        # 2. 获取负荷列名
        all_load_cols, peak_load_cols = get_load_columns()

        # 3. 对2025年的负荷数据进行缩放
        print(f"正在对 '{Config.SHEET_2025}' 的负荷数据进行缩放 (除以10000)...")
        df_2025[all_load_cols] = df_2025[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2025[all_load_cols] = df_2025[all_load_cols] / 10000
        print("缩放完成。")
        
        # 4. 分别为两年的数据计算峰时段负荷占比
        df_2024[all_load_cols] = df_2024[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2025 = calculate_peak_ratio(df_2025, all_load_cols, peak_load_cols)
        df_2024 = calculate_peak_ratio(df_2024, all_load_cols, peak_load_cols)

        # 5. 准备合并数据
        print("\n正在准备数据以便进行年度比较...")
        df_2025['month_day'] = pd.to_datetime(df_2025[Config.DATE_COL], format='%Y%m%d').dt.strftime('%m-%d')
        df_2024['month_day'] = pd.to_datetime(df_2024[Config.DATE_COL], format='%Y%m%d').dt.strftime('%m-%d')

        # 【核心修改】在子集中保留行业编号列
        cols_to_keep = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL, 'month_day', 'peak_ratio']
        df_2025_subset = df_2025[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2025'})
        df_2024_subset = df_2024[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2024'})

        # 6. 合并两年的数据
        # 【核心修改】合并键现在包含行业编号
        merge_keys = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL, 'month_day']
        merged_df = pd.merge(
            df_2025_subset,
            df_2024_subset,
            on=merge_keys,
            how='left'
        )
        merged_df['ratio_2024'] = merged_df['ratio_2024'].fillna(0)
        print("数据合并完成。")

        # 7. 计算最终的差值指标
        merged_df['ratio_diff'] = merged_df['ratio_2025'] - merged_df['ratio_2024']
        print("年度差异计算完成。")

        # 8. 重塑DataFrame以满足最终输出格式
        print("正在生成最终的汇总表...")
        # 【核心修改】将行业编号和名称都作为索引
        final_report = merged_df.pivot_table(
            index=[Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL],
            columns='month_day',
            values='ratio_diff',
            aggfunc='first'
        )
        
        target_dates = ['10-01', '10-02', '10-03', '10-04']
        existing_target_dates = [d for d in target_dates if d in final_report.columns]
        if not existing_target_dates:
            print("[警告] 输出结果为空，因为在数据中找不到10月1日至4日的任何一天。")
            # 创建一个包含正确列名的空DataFrame
            final_columns = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL] + [f"{d.replace('-', '.')}日" for d in target_dates]
            final_report = pd.DataFrame(columns=final_columns)
        else:
            final_report = final_report[existing_target_dates]
            final_report.columns = [f"{col.replace('-', '.')}日" for col in final_report.columns]
        
        # 将多级索引（行业编号和名称）变回普通列
        final_report = final_report.reset_index()

        # 9. 保存结果
        print(f"正在将结果保存到: {Config.OUTPUT_FILE}")
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_report.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')

        print("\n--- 全部任务成功完成！ ---")

    except FileNotFoundError:
        print(f"[致命错误] 无法找到输入文件: {Config.INPUT_FILE}")
    except KeyError as e:
        print(f"[致命错误] 发生列名错误: {e}。请检查Config中的列名是否与Excel文件中的完全一致。")
    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始处理文件: E:\fhzyl.xlsx ---
正在加载Sheet '2025'...
正在加载Sheet '2024'...
正在对 '2025' 的负荷数据进行缩放 (除以10000)...
缩放完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。

正在准备数据以便进行年度比较...
数据合并完成。
年度差异计算完成。
正在生成最终的汇总表...
正在将结果保存到: E:\fhzyl_result.xlsx

--- 全部任务成功完成！ ---


In [6]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    # --- 【请修改】输入和输出路径 ---
    INPUT_FILE = Path(r'E:\fhzyl.xlsx')
    OUTPUT_FILE = Path(r'E:\fhzyl_result1011.xlsx')

    # --- Sheet名称 ---
    SHEET_2025 = '2025'
    SHEET_2024 = '2024'

    # --- 关键列名 ---
    DATE_COL = 'stat_date'
    INDUSTRY_CODE_COL = 'ind_cls_cust' # 【新增】行业编号列
    INDUSTRY_NAME_COL = 'ind_cls_name'

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def get_load_columns() -> tuple[list[str], list[str]]:
    """
    程序化地生成所有负荷列名和峰时段负荷列名。
    【已修正】精确定义峰时段到 load_1500 为止。
    """
    all_load_cols = []
    peak_load_cols = []
    for hour in range(24):
        for minute in range(0, 60, 15):
            time_str = f"{hour:02d}{minute:02d}"
            col_name = f"load_{time_str}"
            all_load_cols.append(col_name)
            
            # 【核心修正】精确判断峰时段
            # 条件：小时在9到14之间（包含），或者小时是15且分钟是0。
            if (hour >= 9 and hour < 15) or (hour == 15 and minute == 0):
                peak_load_cols.append(col_name)
                
    return all_load_cols, peak_load_cols

def calculate_peak_ratio(df: pd.DataFrame, all_load_cols: list[str], peak_load_cols: list[str]) -> pd.DataFrame:
    """
    计算给定DataFrame中每一行的峰时段负荷占比。
    """
    print(f"  -> 正在计算峰时段负荷占比...")
    
    # 1. 确保所有负荷列都是数值类型，非数值转为NaN，然后用0填充
    df[all_load_cols] = df[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # 2. 计算总负荷和峰时段负荷
    total_load = df[all_load_cols].sum(axis=1)
    peak_load = df[peak_load_cols].sum(axis=1)
    
    # 3. 计算占比，并处理总负荷为0的情况（防止除零错误）
    #    使用 np.where: 如果 total_load > 0，则正常计算；否则，占比为0。
    df['peak_ratio'] = np.where(total_load > 0, peak_load / total_load, 0)
    
    print("  -> 计算完成。")
    return df

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"--- 开始处理文件: {Config.INPUT_FILE} ---")

    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    try:
        # 1. 加载数据
        print(f"正在加载Sheet '{Config.SHEET_2025}'...")
        df_2025 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2025)
        
        print(f"正在加载Sheet '{Config.SHEET_2024}'...")
        df_2024 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2024)

        # 2. 获取负荷列名
        all_load_cols, peak_load_cols = get_load_columns()

        # 3. 先聚合，再计算
        group_keys = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL, Config.DATE_COL]
        print("\n正在按天和行业对负荷数据进行预聚合...")
        df_2025[all_load_cols] = df_2025[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2025_agg = df_2025.groupby(group_keys)[all_load_cols].sum().reset_index()
        
        df_2024[all_load_cols] = df_2024[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2024_agg = df_2024.groupby(group_keys)[all_load_cols].sum().reset_index()
        print("预聚合完成。")

        # 4. 对2025年的负荷数据进行缩放
        print(f"正在对 '{Config.SHEET_2025}' 的聚合后负荷数据进行缩放 (除以10000)...")
        df_2025_agg[all_load_cols] = df_2025_agg[all_load_cols] / 10000
        print("缩放完成。")
        
        # 5. 在聚合后的数据上，计算峰时段负荷占比
        df_2025_ratio = calculate_peak_ratio(df_2025_agg, all_load_cols, peak_load_cols)
        df_2024_ratio = calculate_peak_ratio(df_2024_agg, all_load_cols, peak_load_cols)

        # 6. 准备合并数据
        print("\n正在准备数据以便进行年度比较...")
        # 【代码优化】使用 .assign() 避免 PerformanceWarning
        df_2025_ratio = df_2025_ratio.assign(month_day=pd.to_datetime(df_2025_ratio[Config.DATE_COL], format='%Y%m%d').dt.strftime('%m-%d'))
        df_2024_ratio = df_2024_ratio.assign(month_day=pd.to_datetime(df_2024_ratio[Config.DATE_COL], format='%Y%m%d').dt.strftime('%m-%d'))

        cols_to_keep = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL, 'month_day', 'peak_ratio']
        df_2025_subset = df_2025_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2025'})
        df_2024_subset = df_2024_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2024'})

        # 7. 合并两年的数据
        merge_keys = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL, 'month_day']
        merged_df = pd.merge(df_2025_subset, df_2024_subset, on=merge_keys, how='left')
        merged_df['ratio_2024'] = merged_df['ratio_2024'].fillna(0)
        print("数据合并完成。")

        # 8. 计算最终的差值指标
        merged_df['ratio_diff'] = merged_df['ratio_2025'] - merged_df['ratio_2024']
        print("年度差异计算完成。")

        # --- 验算模块 ---
        print("\n--- 开始执行验算模块 ---")
        verification_data = merged_df[(merged_df[Config.INDUSTRY_CODE_COL] == 200) & (merged_df['month_day'] == '10-01')]
        if not verification_data.empty:
            row = verification_data.iloc[0]
            print(f"  行业 '{row[Config.INDUSTRY_NAME_COL]}' (编号 {row[Config.INDUSTRY_CODE_COL]}) 在 10月1日 的计算详情:")
            print(f"    - 2025年峰时段占比 (ratio_2025): {row['ratio_2025']:.10f}")
            print(f"    - 2024年峰时段占比 (ratio_2024): {row['ratio_2024']:.10f}")
            print(f"    - 最终差值 (ratio_diff):       {row['ratio_diff']:.10f}")
            print(f"    - 您手动计算的值进行比较:         -0.026914352")
        else:
            print("  [警告] 在数据中未找到用于验算的 行业200 在 10月1日 的数据。")
        print("--- 验算模块执行完毕 ---\n")

        # 9. 重塑DataFrame
        print("正在生成最终的汇总表...")
        final_report = merged_df.pivot_table(index=[Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL], columns='month_day', values='ratio_diff', aggfunc='first')
        
        target_dates = ['10-01', '10-02', '10-03', '10-04']
        existing_target_dates = [d for d in target_dates if d in final_report.columns]
        if not existing_target_dates:
            print("[警告] 输出结果为空...")
            final_columns = [Config.INDUSTRY_CODE_COL, Config.INDUSTRY_NAME_COL] + [f"{d.replace('-', '.')}日" for d in target_dates]
            final_report = pd.DataFrame(columns=final_columns)
        else:
            final_report = final_report[existing_target_dates]
            final_report.columns = [f"{col.replace('-', '.')}日" for col in final_report.columns]
        
        final_report = final_report.reset_index()

        # 10. 保存结果
        print(f"正在将结果保存到: {Config.OUTPUT_FILE}")
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_report.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')

        print("\n--- 全部任务成功完成！ ---")

    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始处理文件: E:\fhzyl.xlsx ---
正在加载Sheet '2025'...
正在加载Sheet '2024'...

正在按天和行业对负荷数据进行预聚合...
预聚合完成。
正在对 '2025' 的聚合后负荷数据进行缩放 (除以10000)...
缩放完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。

正在准备数据以便进行年度比较...
数据合并完成。
年度差异计算完成。

--- 开始执行验算模块 ---
  行业 '2.林业' (编号 200) 在 10月1日 的计算详情:
    - 2025年峰时段占比 (ratio_2025): 0.2894248859
    - 2024年峰时段占比 (ratio_2024): 0.3163392379
    - 最终差值 (ratio_diff):       -0.0269143520
    - 您手动计算的值进行比较:         -0.026914352
--- 验算模块执行完毕 ---

正在生成最终的汇总表...
正在将结果保存到: E:\fhzyl_result1011.xlsx

--- 全部任务成功完成！ ---


In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    # --- 【请修改】输入和输出路径 ---
    INPUT_FILE = Path(r'E:\fhzyl.xlsx') # <--- 确认你的新文件名
    OUTPUT_FILE = Path(r'E:\fhzyl_t_result.xlsx')

    # --- 【请确认】Sheet名称 ---
    SHEET_2025 = '2025' # <--- 如果新文件中的Sheet名变了，请在这里修改
    SHEET_2024 = '2024' # <--- 如果新文件中的Sheet名变了，请在这里修改

    # --- 【已更新】关键列名 ---
    DATE_COL = 'ds'
    INDUSTRY_NAME_COL = 'code_cls_val_name' 

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def get_load_columns() -> tuple[list[str], list[str]]:
    """
    程序化地生成所有小时负荷列名和峰时段负荷列名。
    """
    all_load_cols = [f"dt_{i}" for i in range(1, 25)]
    # 峰时段: dt_10 (9-10点) 到 dt_15 (14-15点)
    peak_load_cols = [f"dt_{i}" for i in range(10, 16)]
    return all_load_cols, peak_load_cols

def calculate_peak_ratio(df: pd.DataFrame, all_load_cols: list[str], peak_load_cols: list[str]) -> pd.DataFrame:
    """
    计算给定DataFrame中每一行的峰时段负荷占比。
    """
    print(f"  -> 正在计算峰时段负荷占比...")
    df[all_load_cols] = df[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    total_load = df[all_load_cols].sum(axis=1)
    peak_load = df[peak_load_cols].sum(axis=1)
    df['peak_ratio'] = np.where(total_load > 0, peak_load / total_load, 0)
    print("  -> 计算完成。")
    return df

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """【已更新】主执行函数，修正了ds列的日期解析逻辑"""
    print(f"--- 开始处理文件: {Config.INPUT_FILE} ---")

    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    try:
        # 1. 加载数据
        print(f"正在加载Sheet '{Config.SHEET_2025}'...")
        df_2025 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2025)
        
        print(f"正在加载Sheet '{Config.SHEET_2024}'...")
        df_2024 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2024)

        # 2. 获取负荷列名
        all_load_cols, peak_load_cols = get_load_columns()

        # 3. 先聚合，再计算
        group_keys = [Config.INDUSTRY_NAME_COL, Config.DATE_COL]
        print("\n正在按天和行业对负荷数据进行预聚合...")
        df_2025[all_load_cols] = df_2025[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2025_agg = df_2025.groupby(group_keys)[all_load_cols].sum().reset_index()
        
        df_2024[all_load_cols] = df_2024[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2024_agg = df_2024.groupby(group_keys)[all_load_cols].sum().reset_index()
        print("预聚合完成。")

        # 4. 在聚合后的数据上，计算峰时段负荷占比
        df_2025_ratio = calculate_peak_ratio(df_2025_agg, all_load_cols, peak_load_cols)
        df_2024_ratio = calculate_peak_ratio(df_2024_agg, all_load_cols, peak_load_cols)

        # 5. 准备合并数据
        print("\n正在准备数据以便进行年度比较...")
        
        # 【核心修正】在 to_datetime 中明确指定 format='%Y%m%d'
        # 首先将ds列转换为字符串，以防它是整数类型
        df_2025_ratio = df_2025_ratio.assign(month_day=pd.to_datetime(df_2025_ratio[Config.DATE_COL].astype(str), format='%Y%m%d').dt.strftime('%m-%d'))
        df_2024_ratio = df_2024_ratio.assign(month_day=pd.to_datetime(df_2024_ratio[Config.DATE_COL].astype(str), format='%Y%m%d').dt.strftime('%m-%d'))

        cols_to_keep = [Config.INDUSTRY_NAME_COL, 'month_day', 'peak_ratio']
        df_2025_subset = df_2025_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2025'})
        df_2024_subset = df_2024_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2024'})

        # 6. 合并两年的数据
        merge_keys = [Config.INDUSTRY_NAME_COL, 'month_day']
        merged_df = pd.merge(df_2025_subset, df_2024_subset, on=merge_keys, how='left')
        merged_df['ratio_2024'] = merged_df['ratio_2024'].fillna(0)
        print("数据合并完成。")

        # 7. 计算最终的差值指标
        merged_df['ratio_diff'] = merged_df['ratio_2025'] - merged_df['ratio_2024']
        print("年度差异计算完成。")

        # 8. 重塑DataFrame
        print("正在生成最终的汇总表...")
        final_report = merged_df.pivot_table(index=Config.INDUSTRY_NAME_COL, columns='month_day', values='ratio_diff', aggfunc='first')
        
        if final_report.empty:
            print("[警告] 输出结果为空，可能是因为数据中没有任何可供比较的日期。")
            final_report = pd.DataFrame({Config.INDUSTRY_NAME_COL: []})
        else:
            final_report.columns = [f"{col.replace('-', '.')}日" for col in final_report.columns]
        
        final_report = final_report.reset_index()

        # 9. 保存结果
        print(f"正在将结果保存到: {Config.OUTPUT_FILE}")
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_report.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')

        print("\n--- 全部任务成功完成！ ---")

    except FileNotFoundError:
        print(f"[致命错误] 无法找到输入文件: {Config.INPUT_FILE}")
    except KeyError as e:
        print(f"[致命错误] 发生列名错误: {e}。请检查Config中的列名是否与Excel文件中的完全一致。")
    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始处理文件: E:\fhzyl.xlsx ---
正在加载Sheet '2025'...
正在加载Sheet '2024'...

正在按天和行业对负荷数据进行预聚合...
预聚合完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。

正在准备数据以便进行年度比较...
数据合并完成。
年度差异计算完成。
正在生成最终的汇总表...
正在将结果保存到: E:\fhzyl_t_result.xlsx

--- 全部任务成功完成！ ---


In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback
import datetime

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    INPUT_FILE = Path(r'E:\fhzyl.xlsx')
    OUTPUT_FILE = Path(r'E:\fhzyl_new_format_result.xlsx')

    SHEET_2025 = '2025'
    SHEET_2024 = '2024'

    DATE_COL = 'ds'
    INDUSTRY_NAME_COL = 'code_cls_val_name' 

    # --- 【已更新】自我审查模块配置 ---
    # 只需要提供一个基准日期，脚本会自动计算上一年同期
    VERIFICATION_INDUSTRY_NAME = '中成药生产' # <--- 修改为你想要抽查的行业名
    VERIFICATION_BASE_DATE_INT = 20250901   # <--- 修改为你想要抽查的基准日期 (YYYYMMDD格式)

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def get_load_columns() -> tuple[list[str], list[str]]:
    """程序化地生成所有小时负荷列名和峰时段负荷列名。"""
    all_load_cols = [f"dt_{i}" for i in range(1, 25)]
    peak_load_cols = [f"dt_{i}" for i in range(10, 16)]
    return all_load_cols, peak_load_cols

def calculate_peak_ratio(df: pd.DataFrame, all_load_cols: list[str], peak_load_cols: list[str]) -> pd.DataFrame:
    """计算给定DataFrame中每一行的峰时段负荷占比。"""
    print(f"  -> 正在计算峰时段负荷占比...")
    df[all_load_cols] = df[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    total_load = df[all_load_cols].sum(axis=1)
    peak_load = df[peak_load_cols].sum(axis=1)
    df['peak_ratio'] = np.where(total_load > 0, peak_load / total_load, 0)
    print("  -> 计算完成。")
    return df

# --- 【已更新】智能版自我审查函数 ---
def run_verification_module(df_2024_raw: pd.DataFrame, df_2025_raw: pd.DataFrame, final_report: pd.DataFrame):
    """
    执行一次独立的抽样计算，并与最终结果进行比对。
    """
    print("\n--- 开始执行自我审查模块 ---")
    
    try:
        industry_name = Config.VERIFICATION_INDUSTRY_NAME
        base_date_int = Config.VERIFICATION_BASE_DATE_INT
        
        # 自动计算上一年同期日期
        base_date_str = str(base_date_int)
        previous_year_date_int = int(str(int(base_date_str[:4]) - 1) + base_date_str[4:])
        
        all_load_cols, peak_load_cols = get_load_columns()

        print(f"抽样目标: 行业='{industry_name}', 基准日期='{base_date_int}', 同期日期='{previous_year_date_int}'")

        # 1. 计算2025年的占比
        df_2025_sample = df_2025_raw[(df_2025_raw[Config.INDUSTRY_NAME_COL] == industry_name) & (df_2025_raw[Config.DATE_COL] == base_date_int)]
        if df_2025_sample.empty:
            print(f"  [警告] 在{Config.SHEET_2025}数据中未找到日期为 {base_date_int} 的抽样目标，无法进行验算。")
            return
        
        total_load_2025 = df_2025_sample[all_load_cols].sum().sum()
        peak_load_2025 = df_2025_sample[peak_load_cols].sum().sum()
        ratio_2025 = peak_load_2025 / total_load_2025 if total_load_2025 > 0 else 0
        print(f"  [{Config.SHEET_2025}数据] 峰时段负荷: {peak_load_2025:.4f}, 总负荷: {total_load_2025:.4f}, 占比: {ratio_2025:.10f}")

        # 2. 计算2024年的占比
        df_2024_sample = df_2024_raw[(df_2024_raw[Config.INDUSTRY_NAME_COL] == industry_name) & (df_2024_raw[Config.DATE_COL] == previous_year_date_int)]
        if df_2024_sample.empty:
            print(f"  [警告] 在{Config.SHEET_2024}数据中未找到日期为 {previous_year_date_int} 的抽样目标，无法进行验算。")
            return
            
        total_load_2024 = df_2024_sample[all_load_cols].sum().sum()
        peak_load_2024 = df_2024_sample[peak_load_cols].sum().sum()
        ratio_2024 = peak_load_2024 / total_load_2024 if total_load_2024 > 0 else 0
        print(f"  [{Config.SHEET_2024}数据] 峰时段负荷: {peak_load_2024:.4f}, 总负荷: {total_load_2024:.4f}, 占比: {ratio_2024:.10f}")

        # 3. 计算“手动”差值
        manual_diff = ratio_2025 - ratio_2024
        print(f"  [独立计算] 差值 (2025占比 - 2024占比): {manual_diff:.10f}")

        # --- 交叉比对 ---
        date_str_for_lookup = pd.to_datetime(str(base_date_int), format='%Y%m%d').strftime('%m.%d日')
        
        if date_str_for_lookup not in final_report.columns:
            print(f"  [警告] 最终报告中不包含日期 '{date_str_for_lookup}' 的列，无法比对。")
            return
            
        script_result_series = final_report[final_report[Config.INDUSTRY_NAME_COL] == industry_name]
        if script_result_series.empty:
            print(f"  [警告] 最终报告中找不到行业 '{industry_name}' 的行，无法比对。")
            return
            
        script_result = script_result_series[date_str_for_lookup].iloc[0]
        print(f"  [脚本结果] 从最终报告中查询到的差值:   {script_result:.10f}")

        # 4. 输出结论
        if np.isclose(manual_diff, script_result):
            print("  [结论] \033[92m验证通过！\033[0m 独立计算结果与脚本最终结果一致。")
        else:
            print("  [结论] \033[91m验证失败！\033[0m 结果不一致，请检查逻辑。")
            
    except Exception as e:
        print("  [错误] 自我审查模块执行失败。")
        print(traceback.format_exc())
    finally:
        print("--- 自我审查模块执行完毕 ---\n")

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    # ... (main函数的前半部分保持不变) ...
    print(f"--- 开始处理文件: {Config.INPUT_FILE} ---")

    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    try:
        print(f"正在加载Sheet '{Config.SHEET_2025}'...")
        df_2025 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2025)
        
        print(f"正在加载Sheet '{Config.SHEET_2024}'...")
        df_2024 = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_2024)
        
        df_2025_raw_copy = df_2025.copy()
        df_2024_raw_copy = df_2024.copy()

        all_load_cols, peak_load_cols = get_load_columns()

        group_keys = [Config.INDUSTRY_NAME_COL, Config.DATE_COL]
        print("\n正在按天和行业对负荷数据进行预聚合...")
        df_2025[all_load_cols] = df_2025[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2025_agg = df_2025.groupby(group_keys)[all_load_cols].sum().reset_index()
        
        df_2024[all_load_cols] = df_2024[all_load_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_2024_agg = df_2024.groupby(group_keys)[all_load_cols].sum().reset_index()
        print("预聚合完成。")

        df_2025_ratio = calculate_peak_ratio(df_2025_agg, all_load_cols, peak_load_cols)
        df_2024_ratio = calculate_peak_ratio(df_2024_agg, all_load_cols, peak_load_cols)

        print("\n正在准备数据以便进行年度比较...")
        df_2025_ratio = df_2025_ratio.assign(month_day=pd.to_datetime(df_2025_ratio[Config.DATE_COL].astype(str), format='%Y%m%d').dt.strftime('%m-%d'))
        df_2024_ratio = df_2024_ratio.assign(month_day=pd.to_datetime(df_2024_ratio[Config.DATE_COL].astype(str), format='%Y%m%d').dt.strftime('%m-%d'))

        cols_to_keep = [Config.INDUSTRY_NAME_COL, 'month_day', 'peak_ratio']
        df_2025_subset = df_2025_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2025'})
        df_2024_subset = df_2024_ratio[cols_to_keep].rename(columns={'peak_ratio': 'ratio_2024'})

        merge_keys = [Config.INDUSTRY_NAME_COL, 'month_day']
        merged_df = pd.merge(df_2025_subset, df_2024_subset, on=merge_keys, how='left')
        merged_df['ratio_2024'] = merged_df['ratio_2024'].fillna(0)
        print("数据合并完成。")

        merged_df['ratio_diff'] = merged_df['ratio_2025'] - merged_df['ratio_2024']
        print("年度差异计算完成。")

        print("正在生成最终的汇总表...")
        final_report = merged_df.pivot_table(index=Config.INDUSTRY_NAME_COL, columns='month_day', values='ratio_diff', aggfunc='first')
        
        if final_report.empty:
            print("[警告] 输出结果为空...")
            final_report = pd.DataFrame({Config.INDUSTRY_NAME_COL: []})
        else:
            final_report.columns = [f"{col.replace('-', '.')}日" for col in final_report.columns]
        
        final_report = final_report.reset_index()

        # 调用自我审查模块
        run_verification_module(df_2024_raw_copy, df_2025_raw_copy, final_report)

        print(f"正在将结果保存到: {Config.OUTPUT_FILE}")
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_report.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')

        print("\n--- 全部任务成功完成！ ---")

    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始处理文件: E:\fhzyl.xlsx ---
正在加载Sheet '2025'...
正在加载Sheet '2024'...

正在按天和行业对负荷数据进行预聚合...
预聚合完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。
  -> 正在计算峰时段负荷占比...
  -> 计算完成。

正在准备数据以便进行年度比较...
数据合并完成。
年度差异计算完成。
正在生成最终的汇总表...

--- 开始执行自我审查模块 ---
抽样目标: 行业='中成药生产', 日期='20240901'
  [警告] 在2025年数据中未找到抽样目标，无法进行验算。
--- 自我审查模块执行完毕 ---

正在将结果保存到: E:\fhzyl_new_format_result.xlsx

--- 全部任务成功完成！ ---
